<a href="https://colab.research.google.com/github/cmrondon2000/Calculo-volumenes/blob/main/Calculoderrames_VAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==============================================================
# Colab: Volumen derramado por rotura en cada abscisa (con retenciones)
# - Lee un Excel (subido por el usuario) con columnas mínimas: Abscisa, Cota, Diametro, Espesor
# - Espaciamiento entre puntos: se lee desde el archivo (no asumido fijo)
# - Calcula volumen por tramo usando diámetro (pulgadas) y espesor (cm)
# - Para cada abscisa (ruptura hipotética) calcula:
#     * Volumen drenado (m3) y en bbl
#     * Volumen retenido (m3) y en bbl
# - Maneja drenaje parcial de segmentos por interpolación lineal (si un segmento está parcialmente por encima)
# - Usa una Sparse Table (RMQ) para consultas rápidas de máxima elevación entre nodos
# - Salva Excel de salida y un gráfico perfil (Cota vs Abscisa)
# ==============================================================

# ------------- 1) IMPORTS Y PREPARACIÓN -------------
from google.colab import files
import io, os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.ioff()

# ------------- 2) SUBIR ARCHIVO -------------
print("Sube tu archivo Excel (.xls o .xlsx) con columnas: Abscisa, Cota, Diametro, Espesor")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No se subió ningún archivo. Ejecuta la celda nuevamente y sube el archivo.")

# Usamos el primer archivo subido
input_filename = list(uploaded.keys())[0]
print("Archivo subido:", input_filename)

# ------------- 3) LECTURA ROBUSTA DEL EXCEL -------------
def read_excel_flex(bytes_data, filename):
    # intenta lectura automática, luego motores específicos
    try:
        return pd.read_excel(io.BytesIO(bytes_data))
    except Exception as e:
        lower = filename.lower()
        if lower.endswith(".xls"):
            return pd.read_excel(io.BytesIO(bytes_data), engine="xlrd")
        elif lower.endswith(".xlsx"):
            return pd.read_excel(io.BytesIO(bytes_data), engine="openpyxl")
        else:
            return pd.read_excel(io.BytesIO(bytes_data))

df_raw = read_excel_flex(uploaded[input_filename], input_filename)

# Normalizar nombres de columna (strip)
df_raw.columns = [str(c).strip() for c in df_raw.columns]

# ------------- 4) MAPEO DE COLUMNAS (flexible) -------------
# Confirmaste nombres: Abscisa, Cota, Diametro, Espesor
col_map = {}
for col in df_raw.columns:
    lc = col.lower()
    if 'absc' in lc or 'dist' in lc or 'long' in lc:
        col_map['abscisa'] = col
    if 'cot' in lc or 'elev' in lc or 'alt' in lc:
        col_map['cota'] = col
    if 'dia' in lc:
        col_map['diametro'] = col
    if 'esp' in lc or 'thick' in lc:
        col_map['espesor'] = col

# fallback exact
for key in ['abscisa','cota','diametro','espesor']:
    if key not in col_map and key in df_raw.columns:
        col_map[key] = key

required = ['abscisa','cota','diametro','espesor']
missing = [r for r in required if r not in col_map]
if missing:
    raise SystemExit(f"Faltan columnas obligatorias en el archivo: {missing}. Columnas detectadas: {list(df_raw.columns)}")

# ------------- 5) CONSTRUIR DF DE TRABAJO Y CONVERTIR UNIDADES -------------
df = pd.DataFrame()
df['abscisa'] = pd.to_numeric(df_raw[col_map['abscisa']], errors='coerce')
df['cota'] = pd.to_numeric(df_raw[col_map['cota']], errors='coerce')
df['diametro_in'] = pd.to_numeric(df_raw[col_map['diametro']], errors='coerce')  # pulgadas
df['espesor_cm'] = pd.to_numeric(df_raw[col_map['espesor']], errors='coerce')    # cm

# Limpiar filas inválidas y ordenar por abscisa
df = df.dropna(subset=['abscisa','cota']).reset_index(drop=True)
df = df.sort_values('abscisa').reset_index(drop=True)

# Convertir unidades y calcular diámetro interno (m)
df['diametro_m'] = df['diametro_in'] * 0.0254
df['espesor_m'] = df['espesor_cm'] / 100.0
df['diametro_int_m'] = df['diametro_m'] - 2.0 * df['espesor_m']

if (df['diametro_int_m'] <= 0).any():
    raise SystemExit("Error: alguna sección tiene diámetro interno <= 0 tras conversión. Revisa diámetro/espesor.")

# ------------- 5B) OPCIONAL: LECTURA DE VÁLVULAS -------------
# Si el archivo tiene columna "Valvula" o similar (ej: Valve, Válvula, etc.), se toma en cuenta.
# Valores esperados: 1 o 'C' para cerrada, 0 o 'N' para abierta/sin válvula.

valve_col = None
for col in df_raw.columns:
    if 'valv' in col.lower():
        valve_col = col
        break

if valve_col:
    df['valvula'] = df_raw[valve_col].astype(str).str.strip().str.upper()
    df['valvula'] = df['valvula'].replace({'CERRADA':'C','C':'C','1':'C','SI':'C',
                                           'ABIERTA':'N','A':'N','0':'N','NO':'N'})
else:
    df['valvula'] = 'N'  # por defecto, sin válvulas

# Indicar al usuario cuántas válvulas se detectaron
n_valv = (df['valvula'] == 'C').sum()
print(f"✔ Se detectaron {n_valv} válvulas cerradas en el perfil.")


# ------------- 6) DEFINIR TRAMOS (segmentos) -------------
n = len(df)
if n < 2:
    raise SystemExit("Se requieren al menos 2 puntos (abscisas) para definir tramos.")

# Longitud del tramo i -> i+1 (asignada al índice i)
df['seg_length_m'] = df['abscisa'].diff().shift(-1).fillna(0.0)
df['seg_mean_cota'] = (df['cota'] + df['cota'].shift(-1)) / 2.0
df['seg_max_cota'] = np.maximum(df['cota'], df['cota'].shift(-1)).fillna(df['cota'])  # maxima entre nodos
# diámetro interno promedio por tramo
df['seg_diam_int_m'] = ((df['diametro_int_m'] + df['diametro_int_m'].shift(-1)) / 2.0).fillna(df['diametro_int_m'])
# volumen del tramo (m3)
df['seg_vol_m3'] = (np.pi / 4.0) * (df['seg_diam_int_m']**2) * df['seg_length_m']
df['seg_vol_m3'] = df['seg_vol_m3'].fillna(0.0)

total_volume = df['seg_vol_m3'].sum()

# ------------- 7) PRECOMPUTE RMQ (Sparse Table) PARA RANGE MAX (máxima elevación en un tramo de nodos) -------------
node_elev = df['cota'].to_numpy()  # longitud n
LOG = (n).bit_length()
st = [[0]*n for _ in range(LOG)]
for i in range(n):
    st[0][i] = node_elev[i]
j = 1
while (1 << j) <= n:
    length = 1 << j
    half = 1 << (j-1)
    for i in range(0, n - length + 1):
        st[j][i] = max(st[j-1][i], st[j-1][i+half])
    j += 1

def range_max(l, r):
    """Máxima elevación entre nodos l..r inclusive (0-based)."""
    if l > r:
        l, r = r, l
    length = r - l + 1
    if length <= 0:
        return -1e9
    k = (length).bit_length() - 1
    return max(st[k][l], st[k][r - (1 << k) + 1])

# ------------- 8) FUNCIÓN AUXILIAR: fracción del tramo por encima de una cota (interpolación lineal) -------------
def fraction_above(seg_idx, cota_rot):
    """
    Para el segmento entre nodos seg_idx y seg_idx+1,
    devuelve la fracción (0..1) de la longitud del segmento que está por encima de cota_rot,
    asumiendo elevación lineal entre nodos.
    """
    z1 = node_elev[seg_idx]
    z2 = node_elev[seg_idx+1]
    L = df.loc[seg_idx, 'seg_length_m']
    if L == 0:
        return 0.0
    # ambos por encima
    if z1 > cota_rot and z2 > cota_rot:
        return 1.0
    # ambos por debajo o iguales
    if z1 <= cota_rot and z2 <= cota_rot:
        return 0.0
    # hay cruce: calcular fracción por interpolación
    if z2 != z1:
        t = (cota_rot - z1) / (z2 - z1)  # posición param entre 0..1 donde z=cota_rot
        if z2 > z1:
            # ascendente: porción por encima = 1 - t
            return max(0.0, 1.0 - t)
        else:
            # descendente: porción por encima = t
            return max(0.0, t)
    return 0.0

# ------------- 9) CÁLCULO PRINCIPAL: para cada abscisa (ruptura) calcular volúmenes drenados y retenidos -------------
vol_drenado_m3 = np.zeros(n)
vol_drenado_left_m3 = np.zeros(n)
vol_drenado_right_m3 = np.zeros(n)

tol = 1e-9
# Recorremos todas las rupturas posibles (nodos)
for i in range(n):
    cota_rot = node_elev[i]
    vl = 0.0
    vr = 0.0

    # --- Escanear TODOS los segmentos a la izquierda y derecha y decidir si hidráulicamente pueden drenar ---
    # En vez de parar al primer bloqueo, evaluamos cada segmento: se considera que contribuye (parcialmente)
    # si:
    #  1) tiene alguna porción por encima de cota_rot (fraction_above > 0), y
    #  2) el camino entre ese segmento y la rotura no contiene una elevación superior a la elevación máxima del segmento
    #     (es decir, no existe una "cresta" que impida bajar desde el segmento hasta la rotura).
    #
    # Esto permite que, por ejemplo, si la rotura está en el valle más bajo, muchos segmentos aguas arriba contribuyan,
    # siempre que no exista una barrera (cresta) más alta entre ellos y la rotura.

    # izquierda: segmentos indices 0 .. i-1
    for k in range(0, i):
        # comprobar si existe válvula cerrada en los nodos entre el tramo k y la rotura i
        start = k + 1
        end = i - 1
        if start <= end and (df.loc[start:end, 'valvula'] == 'C').any():
            # hay una válvula cerrada en el camino: este tramo no puede drenar hacia la rotura
            continue

        frac = fraction_above(k, cota_rot)
        if frac <= 0.0:
            continue
        seg_max = df.loc[k, 'seg_max_cota']  # máxima elevación entre nodos del tramo
        # máxima elevación en el camino (nodos) entre el tramo y la rotura: nodos k+1 .. i
        max_path = range_max(k+1, i)
        if max_path <= seg_max + tol:
            vl += df.loc[k, 'seg_vol_m3'] * frac
        # si hay cresta mayor, ese tramo no drena hacia la rotura

    # derecha: segmentos indices i .. n-2
    for k in range(i, n-1):
        # comprobar si existe válvula cerrada en los nodos entre la rotura i y el tramo k
        start = i + 1
        end = k
        if start <= end and (df.loc[start:end, 'valvula'] == 'C').any():
            # hay una válvula cerrada en el camino: este tramo no puede drenar hacia la rotura
            continue

        frac = fraction_above(k, cota_rot)
        if frac <= 0.0:
            continue
        seg_max = df.loc[k, 'seg_max_cota']
        # maxima elevación entre nodos i .. k+1
        max_path = range_max(i, k+1)
        if max_path <= seg_max + tol:
            vr += df.loc[k, 'seg_vol_m3'] * frac

    # asignar resultados para la rotura i (asegurar que esto esté dentro del for i)
    vol_drenado_left_m3[i] = vl
    vol_drenado_right_m3[i] = vr
    vol_drenado_m3[i] = vl + vr


# ------------- 10) RESULTADOS Y CONVERSION A BARRILES -------------
df['vol_drenado_left_m3'] = vol_drenado_left_m3
df['vol_drenado_right_m3'] = vol_drenado_right_m3
df['vol_drenado_total_m3'] = vol_drenado_m3
df['vol_retenido_m3'] = total_volume - df['vol_drenado_total_m3']
# Barriles: 1 m3 = 6.2898107714 bbl  (aprox)
M3_TO_BBL = 6.2898107714
df['vol_drenado_total_bbl'] = df['vol_drenado_total_m3'] * M3_TO_BBL
df['vol_retenido_bbl'] = df['vol_retenido_m3'] * M3_TO_BBL
df['vol_total_m3'] = total_volume
df['vol_total_bbl'] = total_volume * M3_TO_BBL

# ------------- 11) (Opcional) añadir columna con segmentos drenados (índices y fracciones) para auditoría -------------
# Construimos listas de segmentos drenados y fracciones (útil para inspección)
drained_segs_list = []
drained_fracs_list = []
for i in range(n):
    segs = []
    fracs = []
    for k in range(0, i):
        frac = fraction_above(k, node_elev[i])
        if frac <= 0: continue
        seg_max = df.loc[k, 'seg_max_cota']
        if range_max(k+1, i) <= seg_max + tol:
            segs.append(str(k))
            fracs.append(f"{frac:.3f}")
    for k in range(i, n-1):
        frac = fraction_above(k, node_elev[i])
        if frac <= 0: continue
        seg_max = df.loc[k, 'seg_max_cota']
        if range_max(i, k+1) <= seg_max + tol:
            segs.append(str(k))
            fracs.append(f"{frac:.3f}")
    drained_segs_list.append(','.join(segs) if segs else 'None')
    drained_fracs_list.append(','.join(fracs) if fracs else 'None')

df['drained_segments'] = drained_segs_list
df['drained_segments_frac'] = drained_fracs_list

# ------------- 12) GUARDAR RESULTADOS EN EXCEL -------------
output_name = "Volumen_Derramado_Por_Abscisa.xlsx"
out_cols = [
    'abscisa','cota','diametro_in','espesor_cm',
    'seg_length_m','seg_mean_cota','seg_diam_int_m','seg_vol_m3',
    'drained_segments','drained_segments_frac',
    'vol_drenado_left_m3','vol_drenado_right_m3','vol_drenado_total_m3','vol_retenido_m3',
    'vol_drenado_total_bbl','vol_retenido_bbl','vol_total_m3','vol_total_bbl'
]
df_out = df[out_cols].copy()
df_out.to_excel(output_name, index=False)
print(f"✔ Archivo de resultados guardado: {output_name}")

# ------------- 13) GRAFICO GENERAL: COTA vs ABSCISA (marcadores coloreados por volumen drenado) -------------
plt.figure(figsize=(12,6))
plt.plot(df['abscisa'], df['cota'], '-o', color='tab:blue', label='Perfil (Cota vs Abscisa)')
sc = plt.scatter(df['abscisa'], df['cota'],
                 s=np.maximum(df['vol_drenado_total_m3'], 0.0) * 15 + 8,
                 c=df['vol_drenado_total_m3'], cmap='plasma', alpha=0.9)
plt.colorbar(sc, label='Volumen drenado (m³)')
plt.xlabel('Abscisa (m)')
plt.ylabel('Cota (m.s.n.m.)')
plt.title('Perfil del ducto (cota vs abscisa) - marcadores ∝ volumen drenado')
plt.grid(True)
summary_plot = "perfil_resumen_volumenes.png"
plt.tight_layout()
plt.savefig(summary_plot, dpi=150)
plt.close()
print(f"✔ Gráfico generado: {summary_plot}")

# ------------- 14) DESCARGAR AUTOMÁTICAMENTE LOS ARCHIVOS EN COLAB -------------
files.download(output_name)
files.download(summary_plot)

# ------------- 15) IMPRESIÓN RÁPIDA DE LOS TOP 10 RESULTADOS -------------
print("\nTop 10 abscisas por volumen drenado (m3):")
display(df_out.sort_values('vol_drenado_total_m3', ascending=False).head(10))

# ----- FIN -----


Sube tu archivo Excel (.xls o .xlsx) con columnas: Abscisa, Cota, Diametro, Espesor


Saving Ole_CNGAR_ARA.xlsx to Ole_CNGAR_ARA.xlsx
Archivo subido: Ole_CNGAR_ARA.xlsx
✔ Se detectaron 3 válvulas cerradas en el perfil.
✔ Archivo de resultados guardado: Volumen_Derramado_Por_Abscisa.xlsx
✔ Gráfico generado: perfil_resumen_volumenes.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Top 10 abscisas por volumen drenado (m3):


,abscisa,cota,diametro_in,espesor_cm,seg_length_m,seg_mean_cota,seg_diam_int_m,seg_vol_m3,drained_segments,drained_segments_frac,vol_drenado_left_m3,vol_drenado_right_m3,vol_drenado_total_m3,vol_retenido_m3,vol_drenado_total_bbl,vol_retenido_bbl,vol_total_m3,vol_total_bbl
264,26400,200.195039,8,0.05,100.0,201.097520,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",292.208992,6.422176,298.631168,2577.058534,1878.333535,16209.210524,2875.689701,18087.544059
197,19700,204.697431,8,0.05,100.0,204.818846,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",274.473919,22.477615,296.951534,2578.738168,1867.768956,16219.775103,2875.689701,18087.544059
194,19400,204.000000,8,0.05,100.0,205.000000,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",266.520289,28.899790,295.420080,2580.269622,1858.136400,16229.407659,2875.689701,18087.544059
233,23300,203.988415,8,0.05,100.0,205.096006,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",265.680104,28.596252,294.276356,2581.413346,1850.942591,16236.601468,2875.689701,18087.544059
231,23100,204.000000,8,0.05,100.0,204.406404,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",259.248808,34.951708,294.200516,2581.489186,1850.465572,16237.078487,2875.689701,18087.544059
265,26500,202.000000,8,0.05,100.0,202.405038,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",289.998285,3.211088,293.209373,2582.480328,1844.231474,16243.312585,2875.689701,18087.544059
198,19800,204.940261,8,0.05,100.0,205.517957,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",273.290484,19.266527,292.557011,2583.132690,1840.128239,16247.415820,2875.689701,18087.544059
266,26600,202.810076,8,0.05,100.0,201.420801,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",284.319671,8.116215,292.435886,2583.253815,1839.366385,16248.177674,2875.689701,18087.544059
209,20900,205.601439,8,0.05,100.0,205.800720,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",272.732646,18.799342,291.531988,2584.157714,1833.681038,16253.863021,2875.689701,18087.544059
260,26000,202.000000,8,0.05,100.0,203.000000,0.2022,3.211088,"0,1,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21...","1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.00...",279.364641,6.422176,285.786816,2589.902885,1797.544996,16289.999063,2875.689701,18087.544059
